In [4]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM, SimpleRNN, Dropout
from tensorflow.keras.callbacks import LambdaCallback

import wandb
from wandb.keras import WandbCallback

import plotutil
from plotutil import PlotCallback

wandb.init()
config = wandb.config

In [5]:
config.repeated_predictions = False
config.look_back = 20

df = pd.read_csv('daily-min-temperatures.csv')
data = df.Temp.astype('float32').values

In [6]:
def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset)-config.look_back-1):
        a = dataset[i:(i+config.look_back)]
        dataX.append(a)
        dataY.append(dataset[i + config.look_back])
    return np.array(dataX), np.array(dataY)

In [7]:
split = int(len(data) * 0.70)
train = data[:split]
test = data[split:]

trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

trainX = trainX[:, :, np.newaxis]
testX = testX[:, :, np.newaxis]

In [9]:
model = Sequential()
model.add(SimpleRNN(1, input_shape=(config.look_back,1 )))

model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(trainX, trainY, epochs=1000, batch_size=1, validation_data=(testX, testY),  callbacks=[WandbCallback(), PlotCallback(trainX, trainY, testX, testY, config.look_back, config.repeated_predictions)])

Train on 2534 samples, validate on 1074 samples
Epoch 1/1000


W0917 21:48:13.158796 140061233088320 callbacks.py:243] Method (on_train_batch_end) is slow compared to the batch update (1.630016). Check your callbacks.


   1/2534 [..............................] - ETA: 1:36:35 - loss: 13.3585 - mae: 13.3585

W0917 21:48:13.185648 140061233088320 callbacks.py:243] Method (on_train_batch_end) is slow compared to the batch update (0.815054). Check your callbacks.


2533/2534 [============================>.] - ETA: 0s - loss: 9.9725 - mae: 9.9724

Error: wandb.Image needs the PIL package. To get it, run "pip install pillow".